In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
import seaborn as sns
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras import Input
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
#Loading datasets

data=pd.read_csv('/content/drive/My Drive/Hackathon Dataset/train.csv')
test=pd.read_csv('/content/drive/My Drive/Hackathon Dataset/test.csv')
#test

In [ ]:
#EDA on the dataset

detected_counts = data.target.value_counts()
plt.figure(figsize = (12,6))
sns.barplot(detected_counts.index, detected_counts.values, alpha = 0.9)
plt.xticks(rotation = 'vertical')
plt.xlabel('Detected classes', fontsize =12)
plt.ylabel('Counts', fontsize = 12)
plt.show()

In [ ]:
#Loading images from the training data and resizing to store the list of pixels and the label

dir='/content/drive/My Drive/Hackathon Dataset/train'
images = []
labels = []
for imagePath,lab in zip(data.Image,data.target):
    image=cv2.imread(os.path.join(dir,imagePath))
    image=cv2.resize(image,(256,256))
    #image=img_to_array(image)
    images.append(image)

    labels.append(lab)

In [ ]:
#Loading images from the test data

dir='/content/drive/My Drive/Hackathon Dataset/test'
imagesT=[]
for imagepath in test.Image :
    image=cv2.imread(os.path.join(dir,imagepath))
    image=cv2.resize(image,(256,256))
    imagesT.append(image)

imagesT = np.array(imagesT, dtype="float") / 255.0

In [ ]:
#Converting the labels using label encoder

le=preprocessing.LabelEncoder()
le.fit(["manipuri","bharatanatyam","odissi","kathakali","kathak","sattriya","kuchipudi","mohiniyattam"])
label=le.transform(labels)
images = np.array(images, dtype="float") / 255.0
label=np.array(label)
#(X_train, X_test, Y_train, Y_test) = train_test_split(images,label, test_size=0.001, random_state=42)
#Y_train=to_categorical(Y_train)
#Y_test=to_categorical(Y_test)
label=to_categorical(label)
X_train=images
Y_train=label

In [ ]:
#Creating ImageDataGenerator object using different parameters and fitting on the train data inorder to dynamically generate additional images while training the model

image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=.05,
    height_shift_range=.075,
    horizontal_flip=True,
    #brightness_range=[0.5,0.9]
)
#training the image preprocessing
image_gen.fit(X_train)

In [ ]:
#Creating a CNN model using pre-trained VGG16 

#new_input = Input(shape=(128, 128, 3))
model=Sequential()
model.add(VGG16(include_top=False, input_shape=(256,256, 3)))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu'))
#model.add(Dense(512,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(8,activation='softmax'))
model.layers[0].trainable=False
#opt = tf.keras.optimizers.RMSprop(learning_rate=1e-3,momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#model.summary()

#model.layers[0].trainable=False
model.summary()

In [ ]:
#Training the model

model.fit_generator(image_gen.flow(X_train, Y_train,batch_size=32),
                     epochs=35)

In [ ]:
#Predicting labels on the test dataset and saving the predictions in the csv file

ynew=model.predict_classes(imagesT)
final=le.inverse_transform(ynew)
final_ans=test.iloc[:,0]
final_ans=pd.DataFrame(final_ans)
final_ans.insert(1,"target",final)
final_ans.to_csv('ouputhopeful7.csv')